In [2]:
!pip install -q gradio
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.7 MB/s eta 0:00:00


In [6]:
!unzip "/content/Aquarium Combined.v2-raw-1024.yolov8.zip" -d /content/aquarium_dataset

Archive:  /content/Aquarium Combined.v2-raw-1024.yolov8.zip
replace /content/aquarium_dataset/README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
from PIL import Image
import os
import torch
from transformers import CLIPProcessor, CLIPModel
import cv2
import numpy as np
import matplotlib.pyplot as plt
import requests
from ultralytics import YOLO
import os
import random
import math
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import gradio as gr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'using {device}')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
using cuda


Functions we need

In [10]:
clip_labels = ['a photo of a fish', 'a photo of a jellyfish', 'a photo of a penguin', 'a photo of a puffin', 'a photo of a shark', 'a photo of a starfish', 'a photo of a stingray']

yolo_model = YOLO("yolov8n.pt")
yolo_model_finetuned = YOLO("best.pt")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def classify_image(image, model_choice):
    image_np = np.array(image)

    if model_choice == "YOLO":
        results = yolo_model(image)
        img_with_boxes = results[0].plot()
        img_with_boxes = Image.fromarray(img_with_boxes[..., ::-1])
        return f"YOLO detection complete ({len(results[0].boxes)} objects)", img_with_boxes

    if model_choice == "YOLO_FINETUNED":
        results = yolo_model_finetuned(image)
        img_with_boxes = results[0].plot()
        img_with_boxes = Image.fromarray(img_with_boxes[..., ::-1])
        return f"YOLO detection complete ({len(results[0].boxes)} objects)", img_with_boxes

    elif model_choice == "CLIP":
        inputs = clip_processor(text=clip_labels, images=image, return_tensors="pt")
        outputs = clip_model(**inputs)
        logits = outputs.logits_per_image
        probs = logits.softmax(dim=1)
        pred_idx = probs.argmax().item()
        pred_label = clip_labels[pred_idx]
        confidence = probs[0, pred_idx].item()
        return f"CLIP prediction: {pred_label} ({confidence*100:.2f}%)", image

    elif model_choice == "YOLO_CLIP":
        results = yolo_model_finetuned(image)
        boxes = results[0].boxes.xyxy
        boxes = boxes.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()
        confidences = results[0].boxes.conf.cpu().numpy()
        img_draw = image_np.copy()

        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = map(int, box)
            crop = Image.fromarray(img_draw[y1:y2, x1:x2])

            inputs = clip_processor(text=clip_labels, images=crop, return_tensors="pt")
            outputs = clip_model(**inputs)
            logits = outputs.logits_per_image
            probs = logits.softmax(dim=1)
            pred_idx = probs.argmax().item()
            pred_label = clip_labels[pred_idx]
            confidence = probs[0, pred_idx].item()

            import cv2
            cv2.rectangle(img_draw, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                img_draw,
                f"{pred_label} {confidence*100:.1f}%",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2
            )

        img_with_labels = Image.fromarray(img_draw)
        return f"CLIP classification on YOLO-detected objects ({len(boxes)} boxes)", img_with_labels

    else:
        return "Invalid model selected", image

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Gradio

In [12]:
device = "cpu"
print(f"Using device: {device}")

demo = gr.Interface(
    fn=classify_image,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Dropdown(choices=["YOLO", "CLIP", "YOLO_CLIP", "YOLO_FINETUNED"], label="Select Model")
    ],
    outputs=[
        gr.Textbox(label="Prediction"),
        gr.Image(label="Output Image")
    ],
    title="Fish Classification Demo",
    description="Upload an image to classify!"
)

demo.launch()


Using device: cpu
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ec072b6f27f93890d7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
